In [5]:
from dataloader import Dataloader
import pandas as pd
import os

pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
# Load Data
path = os.getcwd() + "/../data/"

# load data which creates baskets and coupons
data = Dataloader(path)

In [ ]:
# train-test-split
data.train_test_split()

In [ ]:
# data.create_category_table() 
data.add_categories()

In [ ]:
data.create_feature_dict()

In [ ]:
X_train, y_train, X_test, y_test = data.make_featured_data()

In [ ]:
print(X_train.shape)
X_train.head()

In [ ]:
print(X_test.shape)
X_test.head()